In [ ]:
import json
import numpy as np

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(NpEncoder, self).default(obj)

In [ ]:
try:
print('Processing folder:', folder)
file = "Example-automated.json".format(folder, folder)

with open(file, 'r') as data_file:
    json_data = data_file.read()

data = json.loads(json_data)

# Adding row_range and col_range in the annotation for cells
for i in data:
    if 'properties' in i.keys():
        property_split = sum(sum([[_.split('-')] for _ in i['properties'].split(',')], []), [])
        #print(property_split)
        row1 = int(property_split[0])
        row2 = int(property_split[1])
        #print(i)
        #print(property_split[2])
        col1 = int(property_split[2])
        col2 = int(property_split[3])
        i['row_range'] = [row1, row2]
        i['col_range'] = [col1, col2]
        #print(i)

# Getting the row and col ranges
max_id = max([[int(v) for k, v in i.items() if k == 'id'] for i in data])[0]
row_list = sum([[v.split(',')[0] for k, v in i.items() if k == 'properties'] for i in data], [])
row_max = max([int(_.split('-')[-1]) for _ in row_list])
row_min = min([int(_.split('-')[-1]) for _ in row_list])

col_list = sum([[v.split(',')[1] for k, v in i.items() if k == 'properties'] for i in data], [])
col_max = max([int(_.split('-')[-1]) for _ in col_list])
col_min = min([int(_.split('-')[-1]) for _ in col_list])

print(row_min, row_max, col_min, col_max)


'''col range
- take the min(x1) in the bbox of all row counters
- take the y in the bbox of max row counter 
- max(x2)-min(x1)
- h = y_max-y_min'''

for col_c in range(0, col_max + 1):
    print('the current col under construction is :', col_c)
    col_x1_list = []
    col_x2_list = []

    for row_c in range(0, row_max + 1):
        #print('the current row_c:', row_c)

        for e, i in enumerate(data):
            try:
                if i['properties'] =='{}-{},{}-{}'.format(row_c, row_c, col_c, col_c):
                    #print(i)
                    #print(data[e+1])
                    col_x1_list.append(data[e+1]['bbox'][0])
                    col_x2_list.append(data[e+1]['bbox'][0] + data[e+1]['bbox'][2])

            except KeyError:
                pass

    #print('col_x1_list:', col_x1_list)
    #print('col_x2_list:', col_x2_list)


    if col_x1_list: 
        min_x1 = min(col_x1_list)
        max_x2 = max(col_x2_list)
        print('the min x1 in the column:', min_x1)
        print('the max x2 in the column:', max_x2)
        col_w = max_x2 - min_x1
        print('col_w:', col_w)


    for e, i in enumerate(data):
        try:
            if i['properties'] =='{}-{},{}-{}'.format(row_max, row_max, col_c, col_c): 
                print('max y in col', data[e+1])
                max_y = data[e+1]['bbox'][1]+data[e+1]['bbox'][3]
                print('the y in the bbox of max row counter:', max_y)
            elif i['properties'] =='{}-{},{}-{}'.format(row_min, row_min, col_c, col_c): 
                print('min y in col', data[e+1])
                min_y = data[e+1]['bbox'][1]
                print('the y in the bbox of min row counter:', min_y)

        except KeyError:
            #max_y = 0
            #min_y = 0
            #col_h = max_y - min_y
            pass

    col_h = max_y - min_y
    print('col_h:', col_h)
    print('added:', {
                    "id": max_id + 2,
                    "category": "box",
                    "parent": max_id + 1, 
                    "bbox": [min_x1, min_y, col_w, col_h],
                    "page": 0
                    })


    data.append({
                    "id": max_id + 1,
                    "category": "table_col",
                    "parent": 7,
                    "col_nr": col_c
                    })
    data.append({
                    "id": max_id + 2,
                    "category": "box",
                    "parent": max_id + 1, 
                    "bbox": [min_x1, min_y, col_w, col_h],
                    "page": 0
                    })
    max_id += 3


'''row range
- take the x in the bbox of min row counter 
- take the max(y2) in the bbox of max col counter
- max(x2)-min(x1)
- h = y_max-y_min'''

for row_c in range(0, row_max + 1):
    print('the current row under construction is :', row_c)
    row_y1_list = []
    row_y2_list = []

    for col_c in range(0, col_max + 1):
        print('the current col_c:', col_c)

        for e, i in enumerate(data):
            try:
                if i['properties'] =='{}-{},{}-{}'.format(row_c, row_c, col_c, col_c):
                    print(i)
                    print(data[e+1])
                    row_y1_list.append(data[e+1]['bbox'][1])
                    row_y2_list.append(data[e+1]['bbox'][1] + data[e+1]['bbox'][3])

            except KeyError:
                pass
    print('row_y1_list:', row_y1_list)
    print('row_y2_list:', row_y2_list)

    if row_y1_list: 
        min_y1 = min(row_y1_list)
        max_y2 = max(row_y2_list)
        print('the min y1 in the row:', min_y1)
        print('the max y2 in the row:', max_y2)
        row_h = max_y2 - min_y1
        print('row_h:', row_h)



    for e, i in enumerate(data):
        try:

            if i['properties'] =='{}-{},{}-{}'.format(row_c, row_c, col_max, col_max): 
                print('max x in row')
                print(data[e+1])
                max_x = data[e+1]['bbox'][0] + data[e+1]['bbox'][2]
                print('the x in the bbox of max col counter:', max_x)
            elif i['properties'] =='{}-{},{}-{}'.format(row_c, row_c, col_min, col_min): 
                print('min x in row')
                print(data[e+1])
                min_x = data[e+1]['bbox'][0]
                #print('the y in the bbox of min col counter:', min_x)

        except KeyError:
            #min_x = 0 
            #max_x = 0
            #row_w = max_x - min_x

            pass

    row_w = max_x - min_x
    print('row added:',{
                        "id": max_id + 2,
                        "category": "box",
                        "parent": max_id + 1, 
                        "bbox": [min_x, min_y1, row_w, row_h],
                        "page": 0
                        })    

    data.append({
                    "id": max_id + 1,
                    "category": "table_row",
                    "parent": 7,
                    "row_nr": row_c
                    })
    data.append({
                        "id": max_id + 2,
                        "category": "box",
                        "parent": max_id + 1, 
                        "bbox": [min_x, min_y1, row_w, row_h],
                        "page": 0
                        })



    max_id += 3

print(data[-10:-1])

# Outfile

with open("Example-automated-post.json".\
          format(folder, folder), 'w') as fs:
    json.dump(data, fs, indent=4, cls=NpEncoder)

